# SMOTE 적용

##### **📌 `imblearn` (imbalanced-learn) 패키지**
**주요 기능**
1. **오버샘플링(Over-sampling)** → 소수 클래스 데이터 샘플을 증가시킴  
   - `SMOTE` (Synthetic Minority Over-sampling Technique)  
   - `ADASYN` (Adaptive Synthetic Sampling)  

2. **언더샘플링(Under-sampling)** → 다수 클래스 데이터 샘플을 줄임  
   - `RandomUnderSampler`  
   - `NearMiss`  

3. **앙상블 기법(Ensemble Methods)**  
   - `BalancedBaggingClassifier` → 클래스 균형을 유지하며 배깅 적용  
   - `BalancedRandomForestClassifier` → 불균형 데이터에 최적화된 랜덤 포레스트  

4. **SMOTE 변형 기법**  
   - `BorderlineSMOTE` → 경계선 샘플만 증강  
   - `SMOTENC` → 범주형 데이터를 포함한 SMOTE 

In [ ]:
# !pip install imbalanced-learn

In [ ]:
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification

X, y = make_classification(
    n_classes=2,
    weights=[0.9, 0.1],     # X, y의 비율을 정해줌 
    n_samples=1000,
    random_state=42
)

print(f'Before SMOTE: {np.bincount(y)}') # y에는 정답 데이터가 있으니까 몇 개의 정답 데이터가 있는지 확인해 볼 수 있음

smote = SMOTE(random_state=42)                                    # smote 객체를 만들면서 random_state를 42 줬는데 이건 같은 패턴으로 만들어서 강사님과 비슷한 결과를 볼 수 있게 함
X_resample, y_resample = smote.fit_resample(X, y)                 # fit_resample: 불균형한 데이터셋에서 소수 클래스 데이터를 합성하여 데이터 균형을 맞추는 역할 (새로운 데이터를 생성하여 반환)
print(f'After SMOTE: {np.bincount(y_resample)}') 
print(X.shape, y.shape)                                           # (1000, 20) (1000,): make_classification으로 만들어준 데이터의 개수가 1000개
print(X_resample.shape, y_resample.shape)                         # (1794, 20) (1794,): 새롭게 생성된 데이터가 resample 데이터에 더해짐 (897-103=794 | 1000+794)


Before SMOTE: [897 103]
After SMOTE: [897 897]
(1000, 20) (1000,)
(1794, 20) (1794,)


##### resample하기 전의 데이터 사용

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report       # 평가지표  # sklearn.metrics에는 보통 평가지표 모델이 들어가있음
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)       # stratify: y 데이터의 비율에 맞춰 데이터의 비율을 맞춤

model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# 성능이 고르지 않기 때문에 데이터 샘플에 편향이 있는 데이터 샘플이라는 것을 예측할 수 있음
# 보통 학습 데이터셋에서만 smote를 적용함 -> 이건 원본 데이터에서도 smote를 적용했는데 비교를 보여주기 위해 그런거임

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       179
           1       1.00      0.76      0.86        21

    accuracy                           0.97       200
   macro avg       0.99      0.88      0.93       200
weighted avg       0.98      0.97      0.97       200



##### resample한 데이터 사용

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size=0.2, stratify=y_resample, random_state=0)

model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       180
           1       0.95      0.94      0.95       179

    accuracy                           0.95       359
   macro avg       0.95      0.95      0.95       359
weighted avg       0.95      0.95      0.95       359

